In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
plt.rcParams['figure.figsize'] = [16, 8]
plt.rcParams.update({'font.size': 18})

mat_contents = scipy.io.loadmat(os.path.join('allFaces.mat'))
faces = mat_contents['faces']
m = int(mat_contents['m'])
#print("m", m)
n = int(mat_contents['n'])
#print("n", n)
nfaces = np.ndarray.flatten(mat_contents['nfaces'])

allPersons = np.zeros((n*6,m*6))
count = 0

for j in range(6):
    for k in range(6):
        allPersons[j*n : (j+1)*n, k*m : (k+1)*m] = np.reshape(faces[:,np.sum(nfaces[:count])],(m,n)).T
        count += 1
        
img = plt.imshow(allPersons)
#print(allPersons.shape)
img.set_cmap('gray')
plt.axis('off')
plt.show()

#crop_allPersons = allPersons[0:n,0:m]  #only select one face 

#img = plt.imshow(crop_allPersons)
#img.set_cmap('gray')
#plt.axis('off')
#plt.show()



In [ ]:
allFaces_1 = np.zeros((n*8,m*8,len(nfaces)))
i = 0
for person in range(len(nfaces)):
    subset = faces[:,sum(nfaces[:person]) : sum(nfaces[:(person+1)])]
    allFaces = np.zeros((n*8,m*8))
    count = 0
    
    for j in range(8):
        for k in range(8):
            if count < nfaces[person]:
                allFaces[j*n:(j+1)*n,k*m:(k+1)*m] = np.reshape(subset[:,count],(m,n)).T
                count += 1
              
    allFaces_1[:,:,i] = allFaces 
    i += 1

#select only one person
avgFace = np.mean(allFaces_1[:,:,0],axis=1)
X = allFaces_1[:,:,0] - np.tile(avgFace,(allFaces_1[:,:,0].shape[1],1)).T
print(np.shape(X))
img = plt.imshow(X)
img.set_cmap('gray')
plt.axis('off')
plt.show()

In [ ]:
#compute the economy SVD 

U, S, VT = np.linalg.svd(X,full_matrices=False) #ecomony svd
S_diag = np.diag(S)
#print("S", np.shape(S))
#print("U", U)
print("U shape ", np.shape(U))
#n,m = np.shape(U)


In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(121)
ax1.semilogy(S[:10],'-o',color='r')
plt.title('Singular Values - Ecom SVD')
ax2 = fig1.add_subplot(122)
ax2.plot(np.cumsum(S[:10])/np.sum(S[:10]),'-o',color='b')
plt.title('Singular Values: Cumulative Sum - Ecom SVD')
plt.show()


In [ ]:
#method of the Snapshot 
#take the correlation matrix XX'
X_snap = np.corrcoef(X)

In [ ]:
# Need to find V ans S
U_1, S_snap, VT_snap = np.linalg.svd(X_snap,full_matrices=0)
S_snap_1 = np.diag(S_snap)
#solve for U 
U_snap = X_snap @ VT_snap @ np.transpose(S_snap_1)

In [ ]:
fig2 = plt.figure()
ax1 = fig2.add_subplot(121)
ax1.semilogy(S_snap[:10],'-o',color='r')
plt.title('Singular Values - snap SVD')
ax2 = fig2.add_subplot(122)
ax2.plot(np.cumsum(S_snap[:10])/np.sum(S_snap[:10]),'-o',color='b')
plt.title('Singular Values: Cumulative Sum - snap SVD')
plt.show()

In [ ]:
print(np.shape(U_snap))
print(np.shape(U))

In [ ]:
#compare the first 10 left singular vectors (U) using each method 
fig3 = plt.figure()
ax1 = fig3.add_subplot(121)
img_u1 = ax1.imshow(np.reshape(U[:,0],(m,n)).T)
img_u1.set_cmap('gray')
plt.axis('off')
plt.title('Singular vectors - ecom SVD')

ax2 = fig3.add_subplot(122)
img_u2 = ax2.imshow(np.reshape(U_snap[:,0],(m,n)).T)
img_u2.set_cmap('gray')
plt.axis('off')
plt.title('Singular vectors - snap SVD')

plt.show()